In [1]:
import pandas as pd 
import requests
import os
from unidecode import unidecode
import time

**Getting the list of transfers as well as the schools they're leaving and coming to from https://www.verbalcommits.com/transfers/2021**

In [2]:
#download uncleaned df as csv file so I don't have to keep scraping
if not os.path.exists(os.path.join("data", "uncleaned_transfers.csv")):
    #scrape table from this url
    url = "https://www.verbalcommits.com/transfers/2021"
    #get the page content
    page = requests.get(url)
    #check for status errors
    page.raise_for_status()
    #the desired table is the first table in the list
    df = pd.read_html(page.text)[1]
    #take desired columns
    df = df.drop(columns=["Immediately Eligible", "January Eligible", "Source"])
    #write the dataframe to a csv file so I don't have to keep scraping
    df.to_csv(os.path.join("data", "uncleaned_transfers.csv"), index=False)
#if the file exists print the message
else:
    print("File named uncleaned_transfers.csv already exists")

In [3]:
#read in the csv file just downloaded
transfers = pd.read_csv(os.path.join("data", "uncleaned_transfers.csv"))
#two random digits at the beggining of each height (Ex: 756-8 --> 6-8), so I just take the last 3 digits
transfers["Ht"] = transfers["Ht"].str[-3:]
#height column is of the format LastName FirstNameFirstName LastName so since it's symmetric I can split it in half
#any number multiplied by 2 will always be even so can split it in half
def take_half(entry):
    #find halfway point
    halfway = int(-1*(len(entry) / 2))
    #want the second half of the words as that is "FirstName LastName" format
    return(entry[halfway:])
#apply function to format names correctly
transfers["Name"] = transfers["Name"].apply(take_half)
#there are a bunch of accents above letters that will make it hard to merge with other sources that have stats
#so I'm going to convert it to unidecode as it's the only way I know how to get rid of these
transfers["Name"] = transfers["Name"].apply(unidecode)
#drop na rows (rows where there is no "New School" listed for a player). These players most likely transferred 
#out of division 1 or just returned to the school they were already at
transfers = transfers.dropna()
#visualize the df
transfers.head()

In [4]:
#download cleaned version of the df
if not os.path.exists(os.path.join("data", "transfers.csv")):
    #write cleaned file to csv 
    transfers.to_csv(os.path.join("data", "transfers.csv"), index=False)
#if the csv already exists print the message
else:
    print("The file transfers.csv already exists")

**This is some code I ran for a different project I'm doing. I'm going to comment it out and include it here to show how I got it, but I'm just going to copy the file ("college_advanced.csv") over and filter it to get what I need**

In [5]:
# #This function takes a start_year and end_year and retrieves advanced statistics for all college basketball player
# #seasons in that span among those players that played 75% of their team's games that season. These are compiled and 
# #downloaded to a file called "college_advanced.csv" as the final result

# def get_advanced(start_year=2011, end_year=2022, final_df = pd.DataFrame()):
#     #basketball reference only shows 100 players at a time from "offset" to "offset" + 100
#     #I don't know the max "offset" value, but I do know it goes up by 100 so we'll loop by 100 from 0 to 1000000 (a number that is easily large enough to capture all player)
#     #There is no way there were 1 million players from 2011 to 2022
#     for i in range(0, 1000000, 100):
        
#         #test just 5 loops to start to ensure things go smoothly
#         #if i==500:
#         #    break
         
#         try: #eventually we will get all the players and will reach an "offset" that gives a table that doesn't exist
#             url = f"https://www.sports-reference.com/cbb/play-index/psl_finder.cgi?request=1&match=single&year_min={start_year}&year_max={end_year}&conf_id=&school_id=&class_is_fr=Y&class_is_so=Y&class_is_jr=Y&class_is_sr=Y&pos_is_g=Y&pos_is_f=Y&pos_is_c=Y&games_type=A&qual=pts_per_g&c1stat=&c1comp=gt&c1val=&c2stat=&c2comp=gt&c2val=&c3stat=&c3comp=gt&c3val=&c4stat=&c4comp=gt&c4val=&order_by=bpm&order_by_asc=&offset={i}"
#             print("Visiting: " + url) #use for debugging
#             page = requests.get(url)
#             page.raise_for_status()
#             # get the table with player stats
#             df = pd.read_html(page.text)[0]
#             # returns multi-index dataframe with level 0 being which table and level 1 being the column names
#             # we just want the column names so we need to drop the level 0 column header
#             df = df.droplevel(level=0, axis=1)
#             #drop na rows that come from repeated header level 0 (which table)
#             df = df.dropna()
#             #there are several repeated column header rows, the lines below find those row indices and then drop them
#             repeated_header_indices = list(df[df.Player == "Player"].index)
#             df = df.drop(repeated_header_indices).reset_index(drop=True)
#             #append this table to the final table with all player from start_year to end_year
#             final_df = final_df.append(df)
#             #take 2 seconds between each loop to be curteous and not bombard the website
#             time.sleep(2)
                 
#         except ValueError: #when table doesn't exist a value error will be thrown, break the loop as we have everything
#             print("No more player data to acquire, breaking the loop")
#             break
            
#     #reset the index of the final dataframe
#     final_df = final_df.reset_index(drop=True)
#     #the below lines write the dataframe to a file called filename
#     filename = "college_advanced.csv"
#     if not os.path.exists(os.path.join("data", filename)):
#         final_df.to_csv(os.path.join("data", filename), index=False)
#         return(f"Downloaded file with college players advanced stats from {start_year} to {end_year} to file named {filename}")
#     else:
#         return(f"File {filename} already exists")

**The only people that appear in this dataset are those that played 75% of their team's games (basketball reference's criteria). If the goal is to evaluate what a successful transfer means then I think it's reasonable to use this criteria as it gives a sufficient sample from both seasons.**

In [6]:
#read in the advanced stats dataframe
advanced = pd.read_csv(os.path.join("data", "college_advanced.csv"))
#take only the desired seasons for this project
advanced = advanced[(advanced.Season == "2020-21") | (advanced.Season == "2021-22")]
#drop unnecessary columns
advanced = advanced.drop(columns=["Rk", "Class", "Pos"])
#visualize the dataframe
advanced.head()

,Player,Season,School,Conf,G,MP,PER,TS%,eFG%,ORB%,...,USG%,PProd,ORtg,DRtg,OWS,DWS,WS,OBPM,DBPM,BPM
5,Keegan Murray,2021-22,Iowa,Big Ten,35,1116,37.8,0.638,0.614,9.8,...,29.7,724,134.6,96.8,6.6,2.0,8.7,13.0,2.7,15.7
9,Chet Holmgren,2021-22,Gonzaga,WCC,32,861,31.3,0.691,0.680,8.0,...,21.6,424,127.0,78.7,3.3,3.1,6.3,7.5,7.5,15.0
10,Tari Eason,2021-22,Louisiana State,SEC,33,806,33.4,0.615,0.559,10.5,...,31.8,506,119.2,83.2,3.4,2.5,5.9,9.0,5.7,14.7
15,Walker Kessler,2021-22,Auburn,SEC,34,871,31.4,0.627,0.627,10.9,...,19.0,360,126.0,83.0,2.8,2.7,5.5,6.2,8.0,14.1
18,Luka Garza,2020-21,Iowa,Big Ten,31,975,35.6,0.620,0.596,10.5,...,32.5,655,127.3,100.5,5.5,1.4,6.9,11.9,1.8,13.7


**I'm going to get data from basketball reference for just regular counting stats which I do not have an existing file for already, but I will use the same idea as above with a different url.**

In [7]:
if not os.path.exists(os.path.join("data", "college_regular.csv")):
    
    #initialize the final df to appended to
    final_df = pd.DataFrame()
    #basketball reference only shows 100 players at a time from "offset" to "offset" + 100
    #I don't know the max "offset" value, but I do know it goes up by 100 so we'll loop by 100 from 0 to 1000000 (a number that is easily large enough to capture all player)
    #There is no way there were 1 million players from 2020-21 to 2021-22
    for i in range(0, 1000000, 100):

        #test just 2 loops to start to ensure things go smoothly
#         if i==200:
#             break

        try: #eventually we will get all the players and will reach an "offset" that gives a table that doesn't exist
            url = f"https://www.sports-reference.com/cbb/play-index/psl_finder.cgi?request=1&match=single&year_min=2021&year_max=2022&conf_id=&school_id=&class_is_fr=Y&class_is_so=Y&class_is_jr=Y&class_is_sr=Y&pos_is_g=Y&pos_is_f=Y&pos_is_c=Y&games_type=A&qual=&c1stat=&c1comp=gt&c1val=&c2stat=&c2comp=gt&c2val=&c3stat=&c3comp=gt&c3val=&c4stat=&c4comp=gt&c4val=&order_by=pts_per_g&order_by_asc=&offset={i}"
            print("Visiting: " + url) #use for debugging
            #get the page content
            page = requests.get(url)
            #check for status errors
            page.raise_for_status()
            # get the table with player stats
            df = pd.read_html(page.text)[0]
            # returns multi-index dataframe with level 0 being which table and level 1 being the column names
            # we just want the column names so we need to drop the level 0 column header
            df = df.droplevel(level=0, axis=1)
            #drop na rows that come from repeated header level 0 (which table)
            df = df.dropna()
            #there are several repeated column header rows, the lines below find those row indices and then drop them
            repeated_header_indices = list(df[df.Player == "Player"].index)
            #drop these rows
            df = df.drop(repeated_header_indices).reset_index(drop=True)
            #append this table to the final table with all player from start_year to end_year
            final_df = final_df.append(df)
            #take 2 seconds between each loop to be curteous and not bombard the website
            time.sleep(2)

        except ValueError: #when table doesn't exist a value error will be thrown, break the loop as we have everything
            print("No more player data to acquire, breaking the loop")
            break

    #reset the index of the final dataframe
    final_df = final_df.reset_index(drop=True)
    #write the file to a csv
    final_df.to_csv(os.path.join("data", "college_regular.csv"), index=False)
else:
    print("The file college_regular.csv already exists in the data folder")

The file college_regular.csv already exists in the data folder


In [8]:
#read in the regular stats
regular = pd.read_csv(os.path.join("data", "college_regular.csv"))
#drop overlapping columns that won't be merged on
regular = regular.drop(columns=["Rk", "Class", "Pos", "MP", "Class", "Conf", "G"])
#rename columns as there are 2 MPG columns
regular = regular.rename(columns={"MP.1": "MPG"})
#visualize the data
regular.head()

,Player,Season,School,MPG,FG,FGA,2P,2PA,3P,3PA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,John Meeks,2020-21,Bucknell,31.2,8.5,17.2,6.8,12.8,1.7,4.3,...,7.8,2.0,4.8,6.8,2.3,1.0,0.0,2.7,2.2,25.3
1,Peter Kiss,2021-22,Bryant,35.7,9.1,20.0,7.3,13.7,1.9,6.3,...,6.5,1.4,4.3,5.8,3.2,1.7,0.3,4.2,2.2,25.2
2,Darius McGhee,2021-22,Liberty,33.7,8.4,18.3,4.1,7.3,4.3,11.0,...,4.1,0.5,4.0,4.5,3.6,1.2,0.2,3.0,1.3,24.6
3,Max Abmas,2020-21,Oral Roberts,37.0,8.0,16.8,4.4,8.5,3.6,8.3,...,5.5,0.4,2.8,3.2,3.8,1.5,0.2,2.3,2.2,24.5
4,Luka Garza,2020-21,Iowa,31.5,9.1,16.4,7.6,13.2,1.4,3.2,...,6.4,3.0,5.7,8.7,1.7,0.7,1.6,1.5,2.3,24.1


**Next I will combine the regular and advanced stats and then combine that data with the transfer data**

In [9]:
#merge the advanced stats with the regular stats based on the player name, season, and school
stats = advanced.merge(regular, on=["Player", "Season", "School"])
#rename the column to match up with transfer data set
stats = stats.rename(columns={"Player": "Name"})
#visualize the data
stats.head()

,Name,Season,School,Conf,G,MP,PER,TS%,eFG%,ORB%,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Keegan Murray,2021-22,Iowa,Big Ten,35,1116,37.8,0.638,0.614,9.8,...,5.4,2.9,5.8,8.7,1.5,1.3,1.9,1.1,1.9,23.5
1,Chet Holmgren,2021-22,Gonzaga,WCC,32,861,31.3,0.691,0.680,8.0,...,3.1,1.8,8.1,9.9,1.9,0.8,3.7,1.9,2.7,14.1
2,Tari Eason,2021-22,Louisiana State,SEC,33,806,33.4,0.615,0.559,10.5,...,5.7,2.3,4.3,6.6,1.0,1.9,1.1,2.2,2.8,16.9
3,Walker Kessler,2021-22,Auburn,SEC,34,871,31.4,0.627,0.627,10.9,...,2.6,2.6,5.4,8.1,0.9,1.1,4.6,1.1,2.6,11.4
4,Luka Garza,2020-21,Iowa,Big Ten,31,975,35.6,0.620,0.596,10.5,...,6.4,3.0,5.7,8.7,1.7,0.7,1.6,1.5,2.3,24.1


**This file that I'm downloading ("transfers_with_stats.csv") is the major file that I will be using.**

In [10]:
#merge the combined player stats with the list of transfers based on the name
combined = transfers.merge(stats, on="Name")
#if the file doesn't exist then write the dataframe to a csv, if it does print the message
if not os.path.exists(os.path.join("data", "transfers_with_stats.csv")):
    combined.to_csv(os.path.join("data", "transfers_with_stats.csv"), index=False)
else:
    print("File already exists")

File already exists


# Get 2022-23 Data

**I already have the training data for the 2022-23 season (The 2021-22 season is included in the "transfers_with_stats.csv" data set with the exception of one column. All I need is the post transfer BPM (the BPM for the 2022-23 season) as well as the conference that the player transffered to (2022-23 season conference). I can get both of these things from the advanced stats query on basketball reference. I will also need the list of transfers from the 2021-22 season to the 2022-23 season.** 

**I'm going to be repeating a lot of code (with necessary modifications) here as originally I didn't think I'd be using this year's data. Could easily just create a function like the get_advanced function for getting counting stats and transfer info (it would cut down the number of csv files needed)**

In [11]:
#download uncleaned df as csv file so I don't have to keep scraping
if not os.path.exists(os.path.join("data", "uncleaned_transfers_2023.csv")):
    #scrape table from this url (notice the url has 2022 now instead of 2021)
    url = "https://www.verbalcommits.com/transfers/2022"
    #get the page content
    page = requests.get(url)
    #raise for status errors
    page.raise_for_status()
    #take the first df in the list as that is the one I'm looking for
    df = pd.read_html(page.text)[1]
    #take desired columns
    df = df.drop(columns=["Immediately Eligible", "January Eligible", "Source"])
    #write the df to a csv if the file doesn't already exist
    df.to_csv(os.path.join("data", "uncleaned_transfers_2023.csv"), index=False)
#if the file does exist then print the message 
else:
    print("File named uncleaned_transfers_2023.csv already exists")

File named uncleaned_transfers_2023.csv already exists


In [12]:
# #This function takes a start_year and end_year and retrieves advanced statistics for all college basketball player
# #seasons in that span among those players that played 75% of their team's games that season. These are compiled and 
# #downloaded to a file called "college_advanced_2023.csv" as the final result (only need this season for this project)

def get_advanced(start_year=2011, end_year=2022, final_df = pd.DataFrame()):
    #basketball reference only shows 100 players at a time from "offset" to "offset" + 100
    #I don't know the max "offset" value, but I do know it goes up by 100 so we'll loop by 100 from 0 to 1000000 (a number that is easily large enough to capture all player)
    #There is no way there were 1 million players from 2011 to 2022
    for i in range(0, 1000000, 100):
        
        #test just 5 loops to start to ensure things go smoothly
        #if i==500:
        #    break
         
        try: #eventually we will get all the players and will reach an "offset" that gives a table that doesn't exist
            url = f"https://www.sports-reference.com/cbb/play-index/psl_finder.cgi?request=1&match=single&year_min={start_year}&year_max={end_year}&conf_id=&school_id=&class_is_fr=Y&class_is_so=Y&class_is_jr=Y&class_is_sr=Y&pos_is_g=Y&pos_is_f=Y&pos_is_c=Y&games_type=A&qual=pts_per_g&c1stat=&c1comp=gt&c1val=&c2stat=&c2comp=gt&c2val=&c3stat=&c3comp=gt&c3val=&c4stat=&c4comp=gt&c4val=&order_by=bpm&order_by_asc=&offset={i}"
            print("Visiting: " + url) #use for debugging
            page = requests.get(url)
            page.raise_for_status()
            # get the table with player stats
            df = pd.read_html(page.text)[0]
            # returns multi-index dataframe with level 0 being which table and level 1 being the column names
            # we just want the column names so we need to drop the level 0 column header
            df = df.droplevel(level=0, axis=1)
            #drop na rows that come from repeated header level 0 (which table)
            df = df.dropna()
            #there are several repeated column header rows, the lines below find those row indices and then drop them
            repeated_header_indices = list(df[df.Player == "Player"].index)
            df = df.drop(repeated_header_indices).reset_index(drop=True)
            #append this table to the final table with all player from start_year to end_year
            final_df = final_df.append(df)
            #take 2 seconds between each loop to be curteous and not bombard the website
            time.sleep(2)
                 
        except ValueError: #when table doesn't exist a value error will be thrown, break the loop as we have everything
            print("No more player data to acquire, breaking the loop")
            break
            
    #reset the index of the final dataframe
    final_df = final_df.reset_index(drop=True)
    #the below lines write the dataframe to a file called filename
    filename = "college_advanced_2023.csv"
    #if the file doesn't exist write the 2023 csv file, if it does then print the message
    if not os.path.exists(os.path.join("data", filename)):
        final_df.to_csv(os.path.join("data", filename), index=False)
        return(f"Downloaded file with college players advanced stats from {start_year} to {end_year} to file named {filename}")
    else:
        return(f"File {filename} already exists")

In [13]:
#don't want to re run the code above so I put another if statement here to not call the function if the file exists already
if not os.path.exists(os.path.join("data", "college_advanced_2023.csv")):
    get_advanced(2023, 2023)
else:
    print("File college_advanced_2023.csv already exists in data folder")

File college_advanced_2023.csv already exists in data folder


In [14]:
#read in the transfer data
transfers = pd.read_csv(os.path.join("data", "uncleaned_transfers_2023.csv"))
#two random digits at the beggining of each height (Ex: 756-8 --> 6-8), so I just take the last 3 digits
transfers["Ht"] = transfers["Ht"].str[-3:]
#height column is of the format LastName FirstNameFirstName LastName so since it's symmetric I can split it in half
#any number multiplied by 2 will always be even so can split it in half
def take_half(entry):
    #find halfway point
    halfway = int(-1*(len(entry) / 2))
    #want the second half of the words as that is "FirstName LastName" format
    return(entry[halfway:])
#apply function to format names correctly
transfers["Name"] = transfers["Name"].apply(take_half)
#there are a bunch of accents above letters that will make it hard to merge with other sources that have stats
#so I'm going to convert it to unidecode as it's the only way I know how to get ride of these
transfers["Name"] = transfers["Name"].apply(unidecode)
#drop na rows (rows where there is no "New School" listed for a player)
transfers = transfers.dropna()
#visualize the data frame
transfers.head()

,Stars,Position,Name,Class,Ht,Wt,Previous School,New School
0,2.0,SG,Fletcher Abee,RS SO,6-3,195,The Citadel,UNC Asheville
1,2.0,PF,Riley Abercrombie,RS JR,6-9,220,Rice,Northern Colorado
2,2.0,PF,Micaiah Abii,SO,6-7,250,Liberty,Dallas Baptist
3,2.0,PG,Henry Abraham,RS FR,6-0,175,Eastern Illinois,Coastal Carolina
4,2.0,PF,Linton Acliese III,RS SR,6-6,235,San Francisco State,Eastern Washington
...,...,...,...,...,...,...,...,...
1761,2.0,PG,Levelle Zeigler,SR,6-1,175,Chicago State,Emporia State
1762,2.0,PF,Thomas Zevgaras,SO,6-8,248,Incarnate Word,Newman
1763,2.0,SG,Bogdan Zimonjic,RS FR,6-5,190,Florida Atlantic,Tennessee Tech
1764,3.0,PF,Eric van der Heijden,RS FR,6-9,215,Ole Miss,UNCW


In [15]:
#download cleaned version---write data frame to a csv if the file doesn't exist yet
if not os.path.exists(os.path.join("data", "transfers_2023.csv")):
    transfers.to_csv(os.path.join("data", "transfers_2023.csv"), index=False)
else:
    print("The file transfers_2023.csv already exists")

The file transfers_2023.csv already exists


In [16]:
#read in the downloaded dataframe
advanced = pd.read_csv(os.path.join("data", "college_advanced_2023.csv"))
#take the desired season
advanced = advanced[advanced.Season == "2022-23"]
#drop columns I won't be using 
advanced = advanced.drop(columns=["Rk", "Class", "Pos"])
#visualize the data
advanced.head()

,Player,Season,School,Conf,G,MP,PER,TS%,eFG%,ORB%,...,USG%,PProd,ORtg,DRtg,OWS,DWS,WS,OBPM,DBPM,BPM
0,Trayce Jackson-Davis,2022-23,Indiana,Big Ten,15,466,33.8,0.631,0.610,11.5,...,26.9,282,127.1,91.0,2.2,1.2,3.4,9.8,5.1,15.0
1,Zach Edey,2022-23,Purdue,Big Ten,18,566,39.5,0.650,0.616,20.7,...,31.1,356,131.4,88.3,3.3,1.6,4.9,11.6,2.9,14.5
2,Ja'Vier Francis,2022-23,Houston,AAC,19,260,37.0,0.715,0.736,17.7,...,19.1,115,143.7,72.7,1.3,1.1,2.4,6.6,7.9,14.5
3,Marcus Sasser,2022-23,Houston,AAC,19,576,25.7,0.567,0.528,1.6,...,27.4,297,125.7,84.8,2.5,1.8,4.2,7.9,5.7,13.6
4,Donovan Clingan,2022-23,Connecticut,Big East,20,277,37.7,0.684,0.706,17.8,...,26.1,146,123.8,77.0,1.1,1.1,2.1,6.8,6.7,13.5


**I already have the 2021-22 data as I'm using it to train the model, so just read in the file I created previously in this file.**

In [17]:
#read in the data
advanced2 = pd.read_csv(os.path.join("data", "college_advanced.csv"))
#take the desired season
advanced2 = advanced2[advanced2.Season == "2021-22"]
#drop columns I won't be using 
advanced2 = advanced2.drop(columns=["Rk", "Class", "Pos"])
#visualize the data
advanced2.head()

,Player,Season,School,Conf,G,MP,PER,TS%,eFG%,ORB%,...,USG%,PProd,ORtg,DRtg,OWS,DWS,WS,OBPM,DBPM,BPM
5,Keegan Murray,2021-22,Iowa,Big Ten,35,1116,37.8,0.638,0.614,9.8,...,29.7,724,134.6,96.8,6.6,2.0,8.7,13.0,2.7,15.7
9,Chet Holmgren,2021-22,Gonzaga,WCC,32,861,31.3,0.691,0.680,8.0,...,21.6,424,127.0,78.7,3.3,3.1,6.3,7.5,7.5,15.0
10,Tari Eason,2021-22,Louisiana State,SEC,33,806,33.4,0.615,0.559,10.5,...,31.8,506,119.2,83.2,3.4,2.5,5.9,9.0,5.7,14.7
15,Walker Kessler,2021-22,Auburn,SEC,34,871,31.4,0.627,0.627,10.9,...,19.0,360,126.0,83.0,2.8,2.7,5.5,6.2,8.0,14.1
21,Oscar Tshiebwe,2021-22,Kentucky,SEC,34,1083,35.0,0.626,0.606,19.5,...,24.1,578,128.4,85.0,4.9,3.2,8.0,9.5,3.8,13.3


In [18]:
#want to add 2022-23 season to 2021-22 season
advanced = pd.concat([advanced, advanced2], ignore_index=True)
#visualize the combined data
advanced.head()

,Player,Season,School,Conf,G,MP,PER,TS%,eFG%,ORB%,...,USG%,PProd,ORtg,DRtg,OWS,DWS,WS,OBPM,DBPM,BPM
0,Trayce Jackson-Davis,2022-23,Indiana,Big Ten,15,466,33.8,0.631,0.610,11.5,...,26.9,282,127.1,91.0,2.2,1.2,3.4,9.8,5.1,15.0
1,Zach Edey,2022-23,Purdue,Big Ten,18,566,39.5,0.650,0.616,20.7,...,31.1,356,131.4,88.3,3.3,1.6,4.9,11.6,2.9,14.5
2,Ja'Vier Francis,2022-23,Houston,AAC,19,260,37.0,0.715,0.736,17.7,...,19.1,115,143.7,72.7,1.3,1.1,2.4,6.6,7.9,14.5
3,Marcus Sasser,2022-23,Houston,AAC,19,576,25.7,0.567,0.528,1.6,...,27.4,297,125.7,84.8,2.5,1.8,4.2,7.9,5.7,13.6
4,Donovan Clingan,2022-23,Connecticut,Big East,20,277,37.7,0.684,0.706,17.8,...,26.1,146,123.8,77.0,1.1,1.1,2.1,6.8,6.7,13.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6157,Bakari LaStrap,2021-22,IUPUI,Horizon,28,686,4.8,0.427,0.396,2.3,...,28.1,228,75.4,110.4,-1.4,0.4,-1.0,-7.4,-4.4,-11.8
6158,Noah Bigirumwami,2021-22,Chicago State,WAC,27,372,0.2,0.336,0.337,7.6,...,11.7,50,65.4,113.7,-0.5,0.1,-0.5,-10.3,-1.9,-12.1
6159,Floyd Rideau,2021-22,South Carolina Upstate,Big South,26,295,1.0,0.364,0.348,3.3,...,17.7,59,68.7,109.6,-0.5,0.2,-0.3,-8.8,-3.5,-12.3
6160,Boston Stanton III,2021-22,IUPUI,Horizon,28,703,-0.9,0.372,0.348,0.7,...,19.3,125,60.9,109.8,-1.8,0.4,-1.4,-9.7,-2.9,-12.6


In [19]:
#this cell is to get the regular counting stats for the 2022-23 season
if not os.path.exists(os.path.join("data", "college_regular_2023.csv")):
    
    final_df = pd.DataFrame()
    #basketball reference only shows 100 players at a time from "offset" to "offset" + 100
    #I don't know the max "offset" value, but I do know it goes up by 100 so we'll loop by 100 from 0 to 1000000 (a number that is easily large enough to capture all player)
    #There is no way there were 1 million players from 2020-21 to 2021-22
    for i in range(0, 1000000, 100):

        #test just 2 loops to start to ensure things go smoothly
#         if i==200:
#             break

        try: #eventually we will get all the players and will reach an "offset" that gives a table that doesn't exist (modified years from 2021-2022 to 2022-2023)
            url = f"https://www.sports-reference.com/cbb/play-index/psl_finder.cgi?request=1&match=single&year_min=2022&year_max=2023&conf_id=&school_id=&class_is_fr=Y&class_is_so=Y&class_is_jr=Y&class_is_sr=Y&pos_is_g=Y&pos_is_f=Y&pos_is_c=Y&games_type=A&qual=&c1stat=&c1comp=gt&c1val=&c2stat=&c2comp=gt&c2val=&c3stat=&c3comp=gt&c3val=&c4stat=&c4comp=gt&c4val=&order_by=pts_per_g&order_by_asc=&offset={i}"
            print("Visiting: " + url) #use for debugging
            page = requests.get(url)
            page.raise_for_status()
            # get the table with player stats
            df = pd.read_html(page.text)[0]
            # returns multi-index dataframe with level 0 being which table and level 1 being the column names
            # we just want the column names so we need to drop the level 0 column header
            df = df.droplevel(level=0, axis=1)
            #drop na rows that come from repeated header level 0 (which table)
            df = df.dropna()
            #there are several repeated column header rows, the lines below find those row indices and then drop them
            repeated_header_indices = list(df[df.Player == "Player"].index)
            df = df.drop(repeated_header_indices).reset_index(drop=True)
            #append this table to the final table with all player from start_year to end_year
            final_df = final_df.append(df)
            #take 2 seconds between each loop to be curteous and not bombard the website
            time.sleep(2)

        except ValueError: #when table doesn't exist a value error will be thrown, break the loop as we have everything
            print("No more player data to acquire, breaking the loop")
            break

    #reset the index of the final dataframe
    final_df = final_df.reset_index(drop=True)
    #write data frame to csv file
    final_df.to_csv(os.path.join("data", "college_regular_2023.csv"), index=False)
else:
    print("The file college_regular_2023.csv already exists in the data folder")

The file college_regular_2023.csv already exists in the data folder


In [20]:
#read in the data
regular = pd.read_csv(os.path.join("data", "college_regular_2023.csv"))
#drop overlapping columns that won't be merged on
regular = regular.drop(columns=["Rk", "Class", "Pos", "MP", "Class", "Conf", "G"])
#rename the MP column as their are two of them (one for total minutes and one for MPG)
regular = regular.rename(columns={"MP.1": "MPG"})
regular.head()

,Player,Season,School,MPG,FG,FGA,2P,2PA,3P,3PA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Antoine Davis,2022-23,Detroit Mercy,37.4,8.7,21.3,4.3,10.3,4.4,11.0,...,4.9,0.5,2.7,3.2,3.1,0.9,0.1,3.1,2.3,26.2
1,Peter Kiss,2021-22,Bryant,35.7,9.1,20.0,7.3,13.7,1.9,6.3,...,6.5,1.4,4.3,5.8,3.2,1.7,0.3,4.2,2.2,25.2
2,Darius McGhee,2021-22,Liberty,33.7,8.4,18.3,4.1,7.3,4.3,11.0,...,4.1,0.5,4.0,4.5,3.6,1.2,0.2,3.0,1.3,24.6
3,Antoine Davis,2021-22,Detroit Mercy,37.0,8.2,19.1,4.3,8.9,3.9,10.3,...,4.1,0.5,3.1,3.6,4.4,1.0,0.0,3.3,1.7,23.9
4,Jordan Walker,2022-23,UAB,33.6,7.6,18.2,3.2,7.2,4.4,10.9,...,5.1,0.4,2.2,2.6,4.3,1.5,0.0,3.1,1.8,23.9


In [21]:
#merge the advanced stats with the regular stats based on player, season, and school
stats = advanced.merge(regular, on=["Player", "Season", "School"])
#rename to match up with transfer data set
stats = stats.rename(columns={"Player": "Name"})
#visualize the data
stats.head()

,Name,Season,School,Conf,G,MP,PER,TS%,eFG%,ORB%,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Trayce Jackson-Davis,2022-23,Indiana,Big Ten,15,466,33.8,0.631,0.610,11.5,...,5.5,2.9,6.9,9.8,3.5,0.5,2.9,2.1,1.7,18.6
1,Zach Edey,2022-23,Purdue,Big Ten,18,566,39.5,0.650,0.616,20.7,...,6.8,5.2,7.8,13.0,1.3,0.2,2.3,2.2,1.5,21.3
2,Ja'Vier Francis,2022-23,Houston,AAC,19,260,37.0,0.715,0.736,17.7,...,1.8,2.1,2.7,4.8,0.2,0.6,1.8,0.5,1.9,6.6
3,Marcus Sasser,2022-23,Houston,AAC,19,576,25.7,0.567,0.528,1.6,...,3.7,0.4,2.4,2.8,3.1,1.8,0.2,1.4,1.4,16.9
4,Donovan Clingan,2022-23,Connecticut,Big East,20,277,37.7,0.684,0.706,17.8,...,2.7,2.2,4.2,6.3,0.4,0.6,2.1,1.1,2.2,8.7


In [22]:
#merge the stats data set with the players that are transferring to a new team for the 2022-23 season
combined = transfers.merge(stats, on="Name")
#if the file does NOT exist then write the dataframe to a csv file, else print the appropriate message
if not os.path.exists(os.path.join("data", "transfers_with_stats_2023.csv")):
    #write df to a csv
    combined.to_csv(os.path.join("data", "transfers_with_stats_2023.csv"), index=False)
else:
    print("File named transfers_with_stats_2023.csv already exists in data folder")

File named transfers_with_stats_2023.csv already exists in data folder
